MODULES

In [18]:
from transformers import  AutoTokenizer,VisionEncoderDecoderModel,ViTImageProcessor
from PIL import Image
import torch
from deepface import DeepFace
import json
import cv2
from ultralytics import YOLO
import torchvision
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pymongo import MongoClient
from PIL import Image
import matplotlib.pyplot as plt
import os


In [2]:
#cap
model_name="my_captioning_model"
model1=VisionEncoderDecoderModel.from_pretrained(model_name)
feature_extracter=ViTImageProcessor.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)


#obj
model2 = YOLO("yolov5s.pt")


#rbg
model3 = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True).eval()
"""Run DeepLabV3 to get person mask"""
transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
             ])
#knn
knn = KNeighborsClassifier(n_neighbors=3)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "mymodel_cip"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.48.2"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_r

PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [2]:
from pymongo import MongoClient
client=MongoClient("mongodb://localhost:27017/")
db = client["image_gallery"]
collection=db["images"]

In [ ]:
def load_image(img_path):
    return Image.open(img_path).convert("RGB")

def captioning_model(image):
    pixel_values=feature_extracter(images=image, return_tensors="pt").pixel_values
    #print("pv",pixel_values)
    with torch.no_grad():
     output_ids = model1.generate(pixel_values, max_length=30, num_beams=5,repetition_penalty=2.0,early_stopping=True)
     #print("oid",output_ids)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    #print("oid1",output_ids[0])
    return caption

def analyze_face(image_path):

        return None

def objects_with_count(image_path):
   results = model2(image_path)
   result = results[0]
   objects={}

   for box in result.boxes:
      cls_id = int(box.cls[0])                          
      label = model2.names[cls_id]                 
      conf = float(box.conf[0])                         
      xyxy = box.xyxy[0].tolist()
      if label in objects:
        objects.update({label:objects[label]+1})                      
      else:
        objects[label]=1

   return objects

def get_person_mask(image):
    
    input_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model3(input_tensor)['out'][0]
    output_predictions = output.argmax(0).byte().cpu().numpy()
    return output_predictions

def extract_colors(image, mask, n_colors=3):
    """Extract dominant colors from clothes region"""
    person_pixels = image[mask == 15]  
    if len(person_pixels) == 0:
        return None

    kmeans = KMeans(n_clusters=n_colors)
    kmeans.fit(person_pixels)
    colors = kmeans.cluster_centers_.astype(int)
    return colors

def plot_colors(colors):
    """Plot color boxes"""
    plt.figure(figsize=(8, 2))
    for i, color in enumerate(colors):
        plt.subplot(1, len(colors), i+1)
        plt.axis("off")
        plt.imshow([[color / 255]])
    plt.show()


def euclidean_distance(color1, color2):
   
    return np.sqrt(sum((a - b) ** 2 for a, b in zip(color1, color2)))

def get_color_name(rgb):
    COLOR_NAMES = {
    'red': (255, 0, 0),
    'green': (0, 255, 0),
    'blue': (0, 0, 255),
    'yellow': (255, 255, 0),
    'orange': (255, 165, 0),
    'purple': (128, 0, 128),
    'brown': (165, 42, 42),
    'pink': (255, 192, 203),
    'gray': (169, 169, 169),
    'black': (0, 0, 0),
    'white': (255, 255, 255)
        }
    min_distance = float('inf')
    closest_name = None
    for name, color in COLOR_NAMES.items():
        distance = euclidean_distance(rgb, color)
        if distance < min_distance:
            min_distance = distance
            closest_name = name
    return closest_name



def color(image_path):
  cols=[]
  image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
  mask = get_person_mask(image)
  colors = extract_colors(image, mask)
  #plot_colors(colors)
  for col in colors:
      cols.append(get_color_name(col))
  return cols



def get_img_path(folder_path, valid_extensions=('.jpg','.jpeg','.png')):
    image_paths=[]
    for root, _,files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(valid_extensions):
                image_paths.append(os.path.join(root,file))
    return image_paths


def mapping(image_paths):
       
    db_dict = {}
    
    for image_path in image_paths:
        try:
            # Check if image already exists in collection
            if collection.count_documents({"image_path": image_path}) > 0:
                print(f"Skipping {image_path} - already exists in database")
                continue
                
            # Load and process the image
            image = load_image(image_path)
            
            # Extract various features
            x1 = captioning_model(image)          # Image caption
            x2 = objects_with_count(image_path)   # Object detection with counts
            x3 = color(image_path)               # Color analysis
            x4 = analyze_face(image_path)        # Face analysis
            
            # Compile all features into a dictionary
            final = {
                "image_path": image_path,  # Include image_path in the document
                "caption": x1,
                "objects_count": x2,
                "objects_colors": x3,  # Note: Typo here ('objects_colors' might be intended)
                "actions": x4
            }
            
            db_dict[image_path] = final
            
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")
            continue
    
    return db_dict


    

In [1]:
from pymongo import MongoClient
import numpy as np

client=MongoClient("mongodb://localhost:27017/")
db = client["image_gallery"]
collection=db["images"]

In [6]:
def store_metadata(image_metadata_dict):
    inserted_count = 0
    skipped_count = 0
    
    for image_path, metadata in image_metadata_dict.items():
        if collection.count_documents({"image_path": image_path}) > 0:
            print(f"Image already exists in database: {image_path}")
            skipped_count += 1
            continue
            
        if "objects_colors" in metadata and isinstance(metadata["objects_colors"], np.ndarray):
            metadata["objects_colors"] = metadata["objects_colors"].tolist()
        
        collection.insert_one({
            "image_path": image_path,
            "metadata": metadata 
        })
        inserted_count += 1
    
    print(f"Operation complete. Inserted {inserted_count} new images, skipped {skipped_count} existing ones.")
    return {
        "inserted": inserted_count,
        "skipped": skipped_count,
        "total_in_db": collection.count_documents({})
    }

In [ ]:
image_paths=get_img_path("mygallery")
db_ready=mapping(image_paths)
store_metadata(db_ready)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.



image 1/1 c:\Users\91915\Pernalzing_caption\mygalary\download (1).jpg: 640x640 1 person, 453.4ms
Speed: 30.2ms preprocess, 453.4ms inference, 8.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\91915\Pernalzing_caption\mygalary\download (2).jpg: 544x640 1 person, 451.0ms
Speed: 5.9ms preprocess, 451.0ms inference, 2.6ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 c:\Users\91915\Pernalzing_caption\mygalary\download (3).jpg: 640x640 1 person, 387.8ms
Speed: 4.4ms preprocess, 387.8ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\91915\Pernalzing_caption\mygalary\download (4).jpg: 640x640 1 person, 445.7ms
Speed: 6.1ms preprocess, 445.7ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\91915\Pernalzing_caption\mygalary\download (5).jpg: 640x512 1 person, 351.8ms
Speed: 3.5ms preprocess, 351.8ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 c:\Users

{'inserted': 57, 'skipped': 0, 'total_in_db': 57}

In [5]:
from deepface import DeepFace
x3=analyze_face("mygalary/image copy 2.png")
print(x3)

Action: age:  25%|██▌       | 1/4 [00:01<00:04,  1.40s/it]

Unexpected error: The layer sequential_5 has never been called and thus has no defined input.
None


In [8]:
for i in collection.find():
    print(i)

{'_id': ObjectId('67fd3911c2adb449202137fb'), 'image_path': 'mygalary\\download (1).jpg', 'metadata': {'image_path': 'mygalary\\download (1).jpg', 'caption': 'a man with a beard is smiling ', 'objects_count': {'person': 1}, 'objects_colors': ['gray', 'black', 'pink'], 'actions': {'face_index': 0, 'age': 41, 'gender': 'Man', 'emotion': 'happy'}}, 'embedding': [0.22676491737365723, 0.4237787127494812, 0.1499694436788559, -0.2761893570423126, -0.2687409818172455, -0.10625763982534409, 0.7566846609115601, -0.2759755849838257, 0.05233050882816315, -0.07044953852891922, 0.35892531275749207, -0.5100932717323303, 0.25211775302886963, 0.3374352753162384, -0.25087806582450867, 0.2396567314863205, -0.1857118010520935, 0.2847798764705658, 0.1986674964427948, -0.012338543310761452, -0.3998224437236786, 0.15425541996955872, -0.06289955228567123, -0.3106166422367096, -0.02294626273214817, -0.2559173107147217, 0.0022589806467294693, -0.0742737352848053, 0.20359784364700317, 0.23668766021728516, 0.7100

In [ ]:
import os
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from pymongo import MongoClient
from bson.binary import Binary
import pickle
from PIL import Image, ImageFile
import tkinter as tk
from tkinter import messagebox, simpledialog
from datetime import datetime


ImageFile.LOAD_TRUNCATED_IMAGES = True


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(
    keep_all=True,
    device=device,
    select_largest=False,
    post_process=False
)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)


client = MongoClient("mongodb://localhost:27017/")
db = client["image_gallery"]
verified_faces = db["verified"]
unverified_faces = db["unverified"]

def preprocess_image(img_path):
    """Load and preprocess image with error handling"""
    try:
        img = Image.open(img_path)
        
        if img.mode != 'RGB':
            img = img.convert('RGB')
        return img
    except Exception as e:
        print(f"Error loading {img_path}: {str(e)}")
        return None

def get_face_embedding(img_path):
    """Extract face embedding with better error handling"""
    img = preprocess_image(img_path)
    if img is None:
        return None
    
    try:
        
        faces = mtcnn(img)
        if faces is None:
            print(f"No faces detected in {os.path.basename(img_path)}")
            return None
            
        
        if faces.dim() == 4:  
            faces = [faces[i] for i in range(faces.shape[0])]
        else: 
            faces = [faces]
        
        
        largest_face = max(faces, key=lambda f: f.shape[1]*f.shape[2])
        largest_face = largest_face.unsqueeze(0).to(device)
        
        with torch.no_grad():
            embedding = resnet(largest_face)[0].cpu().numpy()
        
        return embedding
    
    except Exception as e:
        print(f"Error processing {os.path.basename(img_path)}: {str(e)}")
        return None

def find_similar_faces(embedding, threshold=0.7):
    """Find similar faces in database"""
    matches = []
    
    
    for face in verified_faces.find({}):
        try:
            stored_embedding = pickle.loads(face["embedding"])
            similarity = 1 - np.linalg.norm(embedding - stored_embedding)
            if similarity > threshold:
                matches.append({
                    "type": "verified",
                    "id": face["_id"],
                    "name": face["name"],
                    "similarity": similarity,
                    "image_path": face["sample_image"]
                })
        except Exception as e:
            print(f"Error loading face from DB: {str(e)}")
            continue
    
    
    for face in unverified_faces.find({}):
        try:
            stored_embedding = pickle.loads(face["embedding"])
            similarity = 1 - np.linalg.norm(embedding - stored_embedding)
            if similarity > threshold:
                matches.append({
                    "type": "unverified",
                    "id": face["_id"],
                    "similarity": similarity,
                    "image_path": face["image_path"]
                })
        except Exception as e:
            print(f"Error loading unverified face from DB: {str(e)}")
            continue
    
    return sorted(matches, key=lambda x: x["similarity"], reverse=True)

def register_new_face(img_path, embedding, matches):
    """Register a new face with all matching images"""
    root = tk.Tk()
    root.withdraw()
    
    
    message = f"Found {len(matches)+1} similar faces:\n"
    for i, match in enumerate(matches[:3], 1):
        message += f"{i}. {match.get('name', 'Unverified')} ({match['similarity']:.2f})\n"
    
    if messagebox.askyesno("Face Registration", message + "\nRegister this person?"):
        name = simpledialog.askstring("Name", "Enter name for this person:")
        if name:
            
            verified_faces.insert_one({
                "name": name,
                "embedding": Binary(pickle.dumps(embedding)),
                "sample_image": img_path,
                "related_images": [img_path] + [m["image_path"] for m in matches],
                "registration_date": datetime.now(),
                "count": len(matches) + 1
            })
            
            
            for match in matches:
                if match["type"] == "unverified":
                    verified_faces.update_one(
                        {"name": name},
                        {"$push": {"related_images": match["image_path"]}}
                    )
                    unverified_faces.delete_one({"_id": match["id"]})
            
            messagebox.showinfo("Success", f"Registered {name} with {len(matches)+1} images")
            return True
    return False

def process_image_folder(folder_path):
    """Process all images in the folder with better error handling"""
    if not os.path.exists(folder_path):
        print(f"Error: Folder {folder_path} does not exist")
        return
    
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, filename)
            
            
            if verified_faces.count_documents({"$or": [{"sample_image": img_path}, {"related_images": img_path}]}) > 0:
                print(f"Skipping already processed: {filename}")
                continue
            if unverified_faces.count_documents({"image_path": img_path}) > 0:
                print(f"Skipping already processed (unverified): {filename}")
                continue
            
            print(f"\nProcessing {filename}...")
            embedding = get_face_embedding(img_path)
            if embedding is None:
                continue
            
            
            matches = find_similar_faces(embedding)
            
            
            if len(matches) >= 2:  
                if register_new_face(img_path, embedding, matches):
                    continue
            
            
            unverified_faces.insert_one({
                "embedding": Binary(pickle.dumps(embedding)),
                "image_path": img_path,
                "detection_date": datetime.now(),
                "processed": True
            })

def show_stats():
    """Display database statistics"""
    print("\nDatabase Statistics:")
    print(f"Verified faces: {verified_faces.count_documents({})}")
    print(f"Unverified faces: {unverified_faces.count_documents({})}")
    
    print("\nRegistered individuals:")
    for person in verified_faces.find().sort("name", 1):
        print(f"- {person['name']}: {person['count']} images")

if __name__ == "__main__":
    
    gallery_path = "mygallery"  
    
    print("Starting face registration system...")
    process_image_folder(gallery_path)
    show_stats()

C:\Users\91915\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting face registration system...
Skipping already processed (unverified): download (1).jpg
Skipping already processed (unverified): download (2).jpg

Processing download (4).jpg...


In [2]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client["face_gallery"]
collection = db["faces"]
for i in collection.find():
    print(i)

{'_id': ObjectId('67fed028275a4261990e1575'), 'name': 'vijay', 'images': ['mygalary\\download (1).jpg', 'mygalary\\download (2).jpg', 'mygalary\\download (3).jpg', 'mygalary\\download (4).jpg']}
{'_id': ObjectId('67fed028275a4261990e1576'), 'name': 'old', 'images': ['mygalary\\download (5).jpg']}
{'_id': ObjectId('67fed028275a4261990e1577'), 'name': 'nre', 'images': ['mygalary\\download (7).jpg']}
{'_id': ObjectId('67fed028275a4261990e1578'), 'name': 'sdgffs', 'images': ['mygalary\\gettyimages-1252231878-612x612.jpg']}
{'_id': ObjectId('67fed028275a4261990e1579'), 'name': 'sf', 'images': ['mygalary\\gettyimages-1311195899-612x612.jpg']}
{'_id': ObjectId('67fed028275a4261990e157a'), 'name': 'sf', 'images': ['mygalary\\gettyimages-1426540552-612x612.jpg']}
{'_id': ObjectId('67fed028275a4261990e157b'), 'name': 'sf', 'images': ['mygalary\\gettyimages-2097689910-612x612.jpg']}
{'_id': ObjectId('67fed028275a4261990e157c'), 'name': 'fs', 'images': ['mygalary\\gettyimages-517724918-612x612.jpg

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
embedder = SentenceTransformer("paraphrase-MiniLM-L6-v2")

def display_image(image_path):
    """Display image using PIL with error handling"""
    try:
        img = Image.open(image_path)
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.title(os.path.basename(image_path))
        plt.show()
    except Exception as e:
        print(f"Error displaying image {image_path}: {str(e)}")

def is_valid_embedding(embedding):
    """Check if embedding is valid (not NaN and correct shape)"""
    try:
        
        if not isinstance(embedding, np.ndarray):
            embedding = np.array(embedding)
        
        
        if embedding.dtype.kind == 'f':  
            return not np.any(np.isnan(embedding))
        return True
    except Exception as e:
        print(f"Error checking embedding validity: {str(e)}")
        return False

def search_and_show_images(query, top_k=5):
    client=MongoClient("mongodb://localhost:27017/")
    db = client["image_gallery"]
    collection=db["images"]
    """Search images and display results"""
    
    try:
        query_embedding = embedder.encode(query, convert_to_tensor=False)
        if not isinstance(query_embedding, np.ndarray):
            query_embedding = np.array(query_embedding)
    except Exception as e:
        print(f"Error generating query embedding: {str(e)}")
        return []
    
    
    all_images = list(collection.find({}))
    print(all_images)
    
    results = []
    for img in all_images:
        try:
            
            if 'embedding' not in img:
                metadata_text = (
                    f"Caption: {img['metadata']['caption']}. "
                    f"Objects: {img['metadata']['objects_count']}. "
                    f"Colors: {img['metadata']['objects_colors']}. "
                    f"Actions: {img['metadata']['actions']}"
                )
                img_embedding = embedder.encode(metadata_text, convert_to_tensor=False)
                embedding_binary = Binary(pickle.dumps(img_embedding, protocol=2), subtype=128)
                
                
                collection.update_one(
                    {'_id': img['_id']},
                    {'$set': {'embedding': embedding_binary}}
                )
            else:
                img_embedding = np.array(img['embedding'])
            
            
            query_embedding = np.array(query_embedding).reshape(1, -1)
            img_embedding = np.array(img_embedding).reshape(1, -1)
            
            
            sim_score = cosine_similarity(query_embedding, img_embedding)[0][0]
            
            results.append({
                'score': sim_score,
                'image_path': img['image_path'],
                'metadata': img['metadata']
            })
            
        except Exception as e:
            print(f"Error processing image {img.get('image_path', 'unknown')}: {str(e)}")
            continue
    
    if not results:
        print("No valid results found")
        return []
    
    
    results.sort(key=lambda x: x['score'], reverse=True)
    
    
    print(f"\nTop {min(top_k, len(results))} results for: '{query}'")
    for i, result in enumerate(results[:top_k], 1):
        print(f"\nResult {i}:")
        print(f"Score: {result['score']:.4f}")
        print(f"Path: {result['image_path']}")
        print(f"Caption: {result['metadata']['caption']}")
        print(f"objects:{result['metadata']}")
        display_image(result['image_path'])
    
    return results[:top_k]


In [ ]:
import os
from pymongo import MongoClient
import re


client = MongoClient("mongodb://localhost:27017/")
db = client["image_gallery"]
verified_faces = db["verified"]
    
def search_faces(name_query):
    
        
        pattern = re.compile(f".*{re.escape(name_query)}.*", re.IGNORECASE)
        
        
        matching_names = [
            face["name"] 
            for face in verified_faces.find({}, {"name": 1})
            if pattern.match(face["name"])
        ]
        
       
        image_paths = []
        for name in matching_names:
            records = verified_faces.find({"name": name})
            for record in records:
               
                if "sample_image" in record and os.path.exists(record["sample_image"]):
                    image_paths.append(record["sample_image"])
                
                if "related_images" in record:
                    image_paths.extend([
                        img for img in record["related_images"] 
                        if os.path.exists(img)
                    ])
        
        return list(set(image_paths))  



In [ ]:
import numpy as np
from typing import List, Dict, Set

def process_sets(set1: List[Dict], set2: List[List[str]]) -> tuple:

    
    list_path1 = [item['image_path'] for item in set1]
    
    
    list_path2 = [path for sublist in set2 for path in sublist if sublist]
    
    return list_path1, list_path2

def wide_search(paths1: List[str], paths2: List[str]) -> Set[str]:
    return set(paths1).union(set(paths2))

def narrow_search(paths1: List[str], paths2: List[str]) -> Set[str]:
    return set(paths1).intersection(set(paths2))



In [ ]:
from PIL import Image
query = input("\nEnter your search query: ").strip()
set1=search_and_show_images(query)
set2=[]
for token in query.split(" "):
    set2.append(search_faces(token))

list_path1, list_path2 = process_sets(set1, set2)


wide_results = wide_search(list_path1, list_path2)
narrow_results = narrow_search(list_path1, list_path2)

print("List Path 1:", list_path1)
print("List Path 2:", list_path2)
print("\nWide Search Results (Union):", wide_results)
print("Narrow Search Results (Intersection):", narrow_results)



[{'_id': ObjectId('67fd3911c2adb449202137fb'), 'image_path': 'mygalary\\download (1).jpg', 'metadata': {'image_path': 'mygalary\\download (1).jpg', 'caption': 'a man with a beard is smiling ', 'objects_count': {'person': 1}, 'objects_colors': ['gray', 'black', 'pink'], 'actions': {'face_index': 0, 'age': 41, 'gender': 'Man', 'emotion': 'happy'}}, 'embedding': [0.22676491737365723, 0.4237787127494812, 0.1499694436788559, -0.2761893570423126, -0.2687409818172455, -0.10625763982534409, 0.7566846609115601, -0.2759755849838257, 0.05233050882816315, -0.07044953852891922, 0.35892531275749207, -0.5100932717323303, 0.25211775302886963, 0.3374352753162384, -0.25087806582450867, 0.2396567314863205, -0.1857118010520935, 0.2847798764705658, 0.1986674964427948, -0.012338543310761452, -0.3998224437236786, 0.15425541996955872, -0.06289955228567123, -0.3106166422367096, -0.02294626273214817, -0.2559173107147217, 0.0022589806467294693, -0.0742737352848053, 0.20359784364700317, 0.23668766021728516, 0.710

In [17]:
client = MongoClient("mongodb://localhost:27017/")
db = client["image_gallery"]
collection = db["images"]
verified_faces = db["verified"]
unverified_faces = db["unverified"]

In [ ]:
for s in unverified_faces.find():
    print(s)

{'_id': ObjectId('67fec21e5b10990d577edbd8'), 'embedding': b'\x80\x04\x95\x8c\x08\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01M\x00\x02\x85\x94h\x03\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00\x08\x00\x00\xfaJH=\xd9/\xa4\xbc\xeb3\x13=\x98M\xcd<\x88p\x85=\x8b\x0e<\xbd#C\x18<s\xfc\xac\xbc\x8c\xad\xa4\xbc\xd1\xed\xf7\xbc\xb9.\xb4\xbdkN\t\xbd\x10\xd0\xbd\xbd|KL\xbc\xc7=\x96=`N\x01\xbd\x91X9\xbb\x92\x86\xab\xbb\x03\xc2\x19=\x86#G=8\x11\xa3;\xe1\xf3\xbd<v&*\xbdD\'7;\xcdp\xbb<\x88\xd2\xc9\xbb\xc2t\xdb=\x9a1g\xbc\xa7\xb6\x02=\x81Y\xc8\xbd\x0cN_\xbc\x95\xe5.=\x8e\x05\x88<Be\xc2<H\xb7`=^!\xc8;n\xbe^\xbd\xa7\x93\x1e=\x11\xf8\x9d\xbc\xbb\x87\x08=\xce4\xee\xba\xb5@G<\x84{\xa2\xba\x9d8\xf6\xbd\xd3h5\xbc\xf8\x019\xbdP\xbf\xd1<\xc0\x9cA\xbcG~\x17\xbd\x86\x930=_\xb9\xcc=\x95\xcc~==\n8